## 연습문제: 은행 송금


은행은 자신이 계정을 개설하여, 입출금을 할 수 있다.
컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.
주피터 노트북으로 프로그램하세요.

은행 컨트랙의 함수는 다음과 같이 만드세요.

* 입금액을 정해서 입금
	function deposit(uint amount) public payable
* 전액 인출 함수
	function widthdrawAll() public
* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)
	function getBalance() public view returns(uint, uint)
* 다른 계정으로 계좌이체 함수
	function forwardTo(address payable _receiver) public payable



아래 기능을 실행해 보세요.

- 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

## 1단계 : 컨트랙개발

In [1]:
%%writefile src/Bank.sol
pragma solidity ^0.5.0;

contract Bank {
    address payable owner; //sender
    uint balance; //contract balance
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance; //contract balance
    }
    
    // 입금액을 정해서 입금하는 함수
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    
    // 전액 인출 함수
    function widthdrawAll() public {
        owner.transfer(address(this).balance);
    }
    
    // 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)
    function getBalance() public view returns(uint,uint) {
        return (address(this).balance, balance);
    }

    // 다른 계정으로 계좌이체 함수
    function forwardTo(address payable _receiver) public payable {
        _receiver.transfer(333);
    }
}

Overwriting src/Bank.sol


## 2단계 : 컴파일

In [2]:
!solc --abi --bin src/Bank.sol


======= src/Bank.sol:Bank =======
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610218806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf61015a565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101c4565b005b60008047600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f19350505050158015610156573d6000803e3d6000fd5b5050565b6000809054906101000a900473fffffffffffffffffffffffffffffff

In [3]:
!solc --gas src/Bank.sol


======= src/Bank.sol:Bank =======
Gas estimation:
construction:
   41035 + 107200 = 148235
external:
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   widthdrawAll():	infinite


## 3단계 : 컨트랙 배포 - geth로 배포

In [13]:
%%writefile src/BankDeploy.js
//geth
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}
var _abiArray=[{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610218806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf61015a565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101c4565b005b60008047600154915091509091565b8073ffffffffffffffffffffffffffffffffffffffff166108fc61014d9081150290604051600060405180830381858888f19350505050158015610156573d6000803e3d6000fd5b5050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f193505050501580156101c1573d6000803e3d6000fd5b50565b8034146101d057600080fd5b806001600082825401925050819055505056fea265627a7a723158206d0489633afc897071c3f6fca904305ee46670af234d9616f107d010672f8f3f64736f6c63430005100032";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e", gas: 200000, gasPrice: '10000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/BankDeploy.js


In [14]:
!node src/BankDeploy.js

(node:3404) UnhandledPromiseRejectionWarning: Error: The contract code couldn't be stored, please check your gas limit.
    at Object.TransactionError (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-helpers\src\errors.js:93:21)
    at Object.ContractCodeNotStoredError (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-helpers\src\errors.js:102:21)
    at C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-method\src\index.js:394:40
    at processTicksAndRejections (internal/process/task_queues.js:97:5)
(node:3404) UnhandledPromiseRejectionWarning: Unhandled promise rejection. This error originated either by throwing inside of an async function without a catch block, or by rejecting a promise which was not handled with .catch(). To terminate the node process on unhandled promise rejection, use the CLI flag `--unhandled-rejections=strict` (see https://nodejs.org/api/cli.html#cli_unhandled_rejections_mode). (rejection id: 5)
(node:3404) [DEP0018] DeprecationWarning: Unh

## 4단계 : 사용 

In [ ]:
%%writefile src/BankUse.js
//geth
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var abi = [{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getBalanceOfOwner","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var addr = "0xF5cB44f22f4F0e7200197b6Ef03Fd3b25400d781";
var counter = new web3.eth.Contract(abi,addr);
bank.methods.deposit(11111).send({from:"0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e",gas:800000,value:11111});
bank.methods.deposit(222).send({from:"0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e",gas:800000,value:222});
bank.methods.getBalance().call().then(function(value) {console.log(value);});
bank.methods.forwardTo("0xa921C56A63a40EA82D51d6588A2B93e919D71829").send({from:"0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e",gas:100000});
bank.methods.getBalance().call().then(function(value) {console.log(value);});
bank.methods.widthdrawAll().send({from:"0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e",gas:800000});
bank.methods.getBalance().call().then(function(value) {console.log(value);});

In [ ]:
!node src/BankUse.js